# TRTLLM Conversions

In [4]:
import os 
os.mkdir("tllm_checkpoint")

In [7]:
! python3 convert_checkpoint.py --pytorch_ckpt /workspace/CosyVoice/pretrained_models/Fun-CosyVoice3-0.5B/flow.pt 

<frozen importlib._bootstrap_external>:1297: FutureWarning: The cuda.cuda module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.driver module instead.
<frozen importlib._bootstrap_external>:1297: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
2026-01-18 22:55:40,047 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend
/usr/local/lib/python3.12/dist-packages/torch/utils/cpp_extension.py:2330: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
[TensorRT-LLM] TensorRT-LLM version: 0.20.0
CosyVoice PyTorch -> TensorRT-LLM Checkpoint Conversion
  Input:  /workspace/CosyVoice/pretrained_models/Fun-CosyVoice3-0.5B/flow.pt
  Output: tllm_checkpoint
  Dtype:  float16

💾 Saved config t

In [8]:
! trtllm-build \
  --checkpoint_dir tllm_checkpoint \
  --model_cls_file dit_trt.py \
  --model_cls_name CosyVoiceDiT \
  --output_dir ./tllm_engine \
  --max_batch_size 8 \
  --max_seq_len 2000 \
  --remove_input_padding disable --bert_context_fmha_fp32_acc enable


<frozen importlib._bootstrap_external>:1297: FutureWarning: The cuda.cuda module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.driver module instead.
<frozen importlib._bootstrap_external>:1297: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
2026-01-18 22:56:07,992 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend
/usr/local/lib/python3.12/dist-packages/torch/utils/cpp_extension.py:2330: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
[TensorRT-LLM] TensorRT-LLM version: 0.20.0
[01/18/2026-22:56:08] [TRT-LLM] [I] Set bert_attention_plugin to auto.
[01/18/2026-22:56:08] [TRT-LLM] [I] Set gpt_attention_plugin to auto.
[01/18/2026-22:56:08] [TRT-LLM] [I] Set gemm_plugin t

In [9]:
# ! trtllm-build \
#   --checkpoint_dir tllm_checkpoint \
#   --model_cls_file dit_trt.py \
#   --model_cls_name CosyVoiceDiT \
#   --output_dir ./tllm_engine \
#   --max_batch_size 8 \
#   --max_seq_len 2000 \
#   --bert_attention_plugin disable --remove_input_padding disable


# Run inference

In [10]:
import sys 
sys.path.append("/workspace/CosyVoice")
from IPython.display import Audio, display


load hift

In [11]:
import torch 
from hyperpyyaml import load_hyperpyyaml

with open("assets/cosyvoice3_hfgan.yaml", "r") as f:
    configs = load_hyperpyyaml(f)
weights = torch.load('/workspace/CosyVoice/pretrained_models/Fun-CosyVoice3-0.5B/hift.pt', 
                     map_location='cpu')

hift = configs['hift']
hift.load_state_dict(weights)
hift = hift.eval().cuda()

run inference

In [12]:
import time
from trtllm_inference import CosyVoiceDiTTRT
trt_engine = CosyVoiceDiTTRT('tllm_engine',debug_mode=False)

<frozen importlib._bootstrap_external>:1297: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
<frozen importlib._bootstrap_external>:1297: FutureWarning: The cuda.cuda module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.driver module instead.
2026-01-18 22:57:05,506 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend


[TensorRT-LLM] TensorRT-LLM version: 0.20.0


/usr/local/lib/python3.12/dist-packages/torch/utils/cpp_extension.py:2330: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [14]:
mask, mu,  spks, cond = torch.load('assets/example_inputs.pt')
x = torch.zeros(2,80,796).cuda()
updated_x = torch.randn(80, 796).cuda()
x[:] = updated_x
t_span = torch.linspace(0, 1, 10 + 1, device=mu.device, dtype=mu.dtype)
t_span = 1 - torch.cos(t_span * 0.5 * torch.pi)
dt_list = t_span[1:] - t_span[:-1]

start_time = time.time()

for t, dt in zip(t_span[:-1], dt_list):
    t = t * torch.ones(2).cuda()
    out = trt_engine.forward(x, mu, t, spks, cond)
    dphi_dt, cfg_dphi_dt = out[0], out[1]
    dphi_dt = ((1.0 + 0.7) * dphi_dt - 0.7 * cfg_dphi_dt)
    updated_x = updated_x + dt * dphi_dt
    t = t+ dt 
    x[:] = updated_x 
    
end_time = time.time()
print(end_time - start_time)
x = updated_x[:,174:]
tts_speech, _ = hift.inference(speech_feat=x[None], finalize=True)
Audio(tts_speech.cpu().numpy()[0],rate=24000)

0.2277843952178955
